# Negative Words Corpus

In [ ]:
negative = [
    "abandoned", "abuse", "adversity", "agony", "anguish",
    "annoying", "anxiety", "apathy", "betrayal", "bitter",
    "boring", "burden", "chaos", "clumsy", "conflict",
    "coward", "crisis", "cruel", "danger", "darkness",
    "deceit", "defeat", "fuck", "depress", "despair", "despise",
    "destroy", "disgust", "Kamina", "dread", "failure", "fear",
    "foolish", "frustrate", "grief", "guilty", "harm",
    "hate", "hopeless", "hostile", "hurt", "ignorant",
    "inferior", "insecure", "insult", "jealous", "lonely",
    "miserable", "mistake", "negative", "nervous", "pain",
    "panic", "pathetic", "poor", "regret", "rejection",
    "sad", "scared", "shame", "sorrow", "stress",
    "stubborn", "stupid", "suffer", "tears", "tension",
    "threat", "tragic", "ugly", "unhappy", "upset",
    "victim", "waste", "weak", "worry", "kill", "bomb", "hate"
]



# Positive Words Corpus

In [2]:
positive = [
    "adventure", "amazing", "awesome", "beautiful", "brilliant",
    "celebrate", "charming", "delight", "eager", "enjoy",
    "excitement", "fantastic", "friendly", "fun", "happy",
    "harmony", "inspiring", "jubilant", "kindness", "love",
    "marvelous", "optimistic", "passion", "peace", "playful",
    "pleasure", "powerful", "praise", "prosper", "proud",
    "refresh", "rejoice", "relax", "remarkable", "smile",
    "success", "superb", "terrific", "thrilling", "tranquil",
    "vibrant", "victory", "wonderful", "cheerful", "blessed",
    "grateful", "grace", "imagine", "vibrant", "glorious",
    "extraordinary", "radiant", "bliss", "ecstatic", "hope",
    "admirable", "caring", "creative", "freedom", "healing",
    "loving", "nurturing", "paradise", "respect", "wholesome", "good",
    "love", "thankyou", "graceful"
]

# Simulated training Audio File

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import librosa

In [2]:
# Sample data (replace with your labeled audio data)
data = [
    {'audio_file': 'positive.wav', 'sentiment': 'positive'},
    {'audio_file': 'negative.wav', 'sentiment': 'negative'},
    
    # Add more data entries
]

# Convert sentiment labels to numerical values (e.g., 0 for negative, 1 for neutral, 2 for positive)
sentiments = {'negative': 0, 'neutral': 1, 'positive': 2}


In [3]:
data

[{'audio_file': 'positive.wav', 'sentiment': 'positive'},
 {'audio_file': 'negative.wav', 'sentiment': 'negative'}]

In [4]:
# Feature extraction function using MFCC
def extract_features(audio_file):
    y, sr = librosa.load(audio_file, mono=True, duration=5)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfccs


In [5]:
# Prepare data for training
X = []  # Features (MFCCs)
y = []  # Sentiment labels

for entry in data:
    mfccs = extract_features(entry['audio_file'])
    X.append(mfccs)
    y.append(sentiments[entry['sentiment']])

SystemError: initialization of _internal failed without raising an exception

In [6]:
X = np.array(X)
y = np.array(y)

# Normalize the data
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - mean) / std


C:\Users\rahul\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rahul\anaconda3\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\rahul\anaconda3\lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\rahul\anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\rahul\anaconda3\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
print(X_train)

In [8]:
print(X_test)

NameError: name 'X_test' is not defined

In [9]:
# Build an LSTM-based model
model = keras.Sequential([
    keras.layers.LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(3, activation='softmax')  
])


NameError: name 'X_train' is not defined

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy[1]}")

NameError: name 'model' is not defined

# Speech Recognition And check Sentiment of call audio

In [3]:
import speech_recognition as sr

# Function to predict sentiment from user speech input
def predict_sentiment():
    r = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Speak something...")
        audio = r.listen(source)
        
    try:
        text = r.recognize_google(audio)
        print("You said:", text)

        # Tokenize the recognized text into words
        words = text.lower().split()
       
        # Determine sentiment based on word counts
        predictions = model.predict(np.expand_dims(mfccs, axis=0))
        positive_ = sum(1 for word in words if word in positive)
        negative_ = sum(1 for word in words if word in negative)

        # Map the prediction to a sentiment label (e.g., negative, neutral, positive)
        sentiment_label = np.argmax(predictions)

        sentiments = {0: 'negative', 1: 'neutral', 2: 'positive'}
        predicted_sentiment = sentiments[sentiment_label]

        
        if positive_ > negative_:
            sentiment = "Positive"
        elif negative_ > positive_:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"

        return sentiment
    except sr.UnknownValueError:
        return "Sorry, I could not understand what you said."
    except sr.RequestError:
        return "Sorry, there was an error with the speech recognition service."



In [4]:
n = int(input("Enter the number of times to predict sentiment: "))

for i in range(n):
    result = predict_sentiment()
    print(f"Prediction {i + 1}: {result}")

Enter the number of times to predict sentiment: 1
Speak something...
You said: hello hello hello hello hello hello


NameError: name 'model' is not defined